## Import Required Libraries ##

In [1]:
import os
import dotenv
import google.generativeai as genai
dotenv.load_dotenv()
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

c:\Users\User\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Game of Throne Book using PyPDF ##

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Books/got.pdf")
data = loader.load()

In [3]:
print(f"Total pages: {len(data)}")

Total pages: 755


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total number of documents:", len(docs))

Total number of documents: 2249


### Look into the content of docs ###

In [5]:
for doc in docs[3:5]: #for slice 3-4

    print(f"Page no. {doc.metadata['page']}:\n{doc.page_content}")

Page no. 5:
previous | Table of Contents | next
PROLOGUE
We should start back,” Gared urged as the woods began to grow dark around them. “The 
wildlings are dead.”
“Do the dead frighten you?” Ser Waymar Royce asked with just the hint of a smile.
Gared did not rise to the bait. He was an old man, past fifty, and he had seen the 
lordlings come and go. “Dead is dead,” he said. “We have no business with the dead.”
“Are they dead?” Royce asked softly. “What proof have we?”
“Will saw them,” Gared said. “If he says they are dead, that’s proof enough for me.”
Will had known they would drag him into the quarrel sooner or later. He wished it had 
been later rather than sooner. “My mother told me that dead men sing no songs,” he put 
in.
“My wet nurse said the same thing, Will,” Royce replied. “Never believe anything you 
hear at a woman’s tit. There are things to be learned even from the dead.” His voice 
echoed, too loud in the twilit forest.
Page no. 5:
hear at a woman’s tit. There are things

### Loading Genai's Text-embedding-004 model ###

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")
print(f"Lenght of embedding vector :{len(vector)}")

Lenght of embedding vector :768


### Loading Pinecone and Creating vectorstore ###

In [7]:
from langchain_pinecone import PineconeVectorStore

index_name = "harrypotter-qna"

vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={'k': 5})

retrieved_docs = retriever.invoke("What is kings landing?")

In [9]:
len(retrieved_docs)

5

In [10]:
for doc in retrieved_docs:
    print("-----------")
    print(doc.page_content)

-----------
High overhead, the far-eyes sang out from the rigging. Captain Moreo came scrambling 
across the deck, giving orders, and all around them the Storm Dancer burst into frenetic 
activity as King’s Landing slid into view atop its three high hills.
Three hundred years ago, Catelyn knew, those heights had been covered with forest, and 
only a handful of fisherfolk had lived on the north shore of the Blackwater Rush where 
that deep, swift river flowed into the sea. Then Aegon the Conqueror had sailed from 
Dragonstone. It was here that his army had put ashore, and there on the highest hill that 
he built his first crude redoubt of wood and earth.
Now the city covered the shore as far as Catelyn could see; manses and arbors and 
granaries, brick storehouses and timbered inns and merchant’s stalls, taverns and 
graveyards and brothels, all piled one on another. She could hear the clamor of the fish 
market even at this distance. Between the buildings were broad roads lined with tr

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.3, max_tokens=500)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three to five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [14]:
response = rag_chain.invoke({"input": "who killed robbert baratheon?"})
print(response["answer"])

King Robert Baratheon was killed by a boar during a hunt. However, it is implied that the wine he drank before the hunt may have been poisoned, leading to speculation about who was responsible for his death. Varys wonders who gave the king the wine. Lysa named Cersei in the letter she sent to Winterfell, but now she seemed certain that Tyrion was the killer.


In [15]:
response = rag_chain.invoke({"input": "who killed robbert baratheon? how did he die?"})
print(response["answer"])

King Robert Baratheon died while hunting a boar. Some stories say he was killed by the boar, while others suggest he died from being poisoned by Varys the Spider or the queen. Another story says he died from choking on a fish bone or a pox. The king himself blamed the wine he drank while hunting.


In [16]:
response = rag_chain.invoke({"input": "who is next king after robbert"})
print(response["answer"])

According to Robb, Joffrey is Robert's eldest trueborn son, making him the rightful heir to the throne by the laws of the realm. If Joffrey were to die, his younger brother Tommen would be next in line to be king.
